In [1]:
with open('data/airbus_report.xhtml') as f:
    report = f.read()

report[:1000]

'<?xml version="1.0" encoding="UTF-8"?><html xmlns="http://www.w3.org/1999/xhtml" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:utr="http://www.xbrl.org/2009/utr" xmlns:num="http://www.xbrl.org/dtr/type/numeric" xmlns:nonnum="http://www.xbrl.org/dtr/type/non-numeric" xmlns:xbrldt="http://xbrl.org/2005/xbrldt" xmlns:ref="http://www.xbrl.org/2006/ref" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:airbus="http://www.airbus.com" xml:lang="en" xmlns:esef_cor="https://www.esma.europa.eu/taxonomy/2022-03-24/esef_cor" xmlns:ifrs-full="https://xbrl.ifrs.org/taxonomy/2022-03-24/ifrs-full"><head><title></title><style type="text/css">#text

In [10]:
from html.parser import HTMLParser
from collections import Counter, defaultdict

class ReportParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.tags = defaultdict(list)
        self.xbrlTags = []
        self.contexts = []
        self.context = []
        self.datas = []

    def get_current_context(self):
        def process(item):
            res = item['tag']
            if 'class' in item.keys():
                res += f' {item['class']}'
            return res
        
        return " >> ".join([process(item) for item in self.context])
        
    def handle_starttag(self, tag, attr):
        attr = dict(attr)
        curr = {
            'tag': tag, 
        }
        if tag not in ['html', 'body']:
            curr['attr'] = attr
            curr['class'] = attr.get('class', '')
            
        self.context.append(curr)
        
        if tag.startswith('ix') or tag.startswith('xbrl'):
            self.tags[tag].append(attr)
            if tag.startswith('ix'):
                xbrlTag = attr.get('name', '')
                self.xbrlTags.append(xbrlTag)
                cxt = self.get_current_context()
                self.contexts.append(cxt)
            
    def handle_endtag(self, tag):
        self.context.pop(-1)
        pass
        
    def handle_data(self, data):
        if self.context[-1]['tag'] not in ['style'] and not self.context[-1]['tag'].startswith('ix') and not self.context[-1]['tag'].startswith('xbrl'):
            if 'data' in self.context[-1].keys():
                self.context[-1]['data'] += data
            else:
                self.context[-1]['data'] = data
            self.datas.append(data)

parser = ReportParser()
parser.feed(report)
# for context in parser.contexts[:150]:
#     print(context)
#     print('-------------')
# for k, v in parser.tags.items():
#     print(f'{k}: {len(v)}')
#     print(f'{v[0]}\n')
# Counter(parser.xbrlTags)
# Counter(parser.contexts)
raw = "".join(parser.datas)

print(raw)


  
Airbus SE Report of the Board of Directors 2024 (Issued as of 19 February 2025) Contents 1 Information on the Company’s Activities 3 1.1 Overview 3 1.2 Airbus (Commercial Aircraft) 8 1.3 Airbus Helicopters 20 1.4 Airbus Defence and Space 27 1.5 Other Corporate Activities 38 2 Corporate Governance 47 2.1 Board of Directors 47 2.2 Executive Committee 59 2.3 Remuneration Report 60 2.4 Dutch corporate governance code 78 2.5 Miscellaneous 80 3 Risk Management and Internal Controls 81 3.1 Risk factors 81 3.2 Enterprise Risk Management System 96 3.3 Insurance 99 3.4 Internal Control 100 3.5 Internal Audit 101 4 General Information on the Company and its Share Capital 102 4.1 Description of the Share Capital 102 4.2 Shareholding and voting rights 104 4.3 Share price performance 2024 115 4.4 Dividends 116 4.5 Taxation 117 5 Financial Information 119 5.1 Financial Performance 120 5.2 Information on Airbus SE auditors 125 6 Non-Financial Information: Sustainability Statement 126 6.1  General 